In [ ]:
!pip install rank-bm25
!pip install PyPDF2
!pip install faiss-cpu
from huggingface_hub import login
#login(token="")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 39.8 MB/s eta 0:00:00


In [5]:
"""
Système RAG Hybride pour l'Analyse de Documents d'Assurance
Phase 1: Extraction et Prétraitement du Texte depuis PDF
"""
# Installer la bibliothèque
import PyPDF2


import re
import json
from pathlib import Path
from typing import List, Dict, Optional
from dataclasses import dataclass, asdict
import unicodedata


@dataclass
class DocumentSection:

    id: str
    title: str
    content: str
    section_type: str
    page_number: int
    parent_section: Optional[str] = None
    metadata: Optional[Dict] = None

class PDFExtractor:


    def __init__(self, pdf_path: str):
        self.pdf_path = Path(pdf_path)
        self.text = ""
        self.page_count = 0

    def extract_text(self) -> str:
        """Extrait le texte complet du PDF"""
        if not self.pdf_path.exists():
            raise FileNotFoundError(f"❌ Fichier non trouvé: {self.pdf_path}")

        print(f"📖 Lecture du PDF: {self.pdf_path.name}")

        try:
            with open(self.pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                self.page_count = len(pdf_reader.pages)

                print(f"   📄 {self.page_count} pages détectées")

                text_parts = []
                for i, page in enumerate(pdf_reader.pages, 1):
                    page_text = page.extract_text()
                    if page_text:
                        text_parts.append(f"\n--- PAGE {i} ---\n{page_text}")

                    if i % 10 == 0:
                        print(f"   ⏳ Extraction en cours... {i}/{self.page_count} pages")

                self.text = "\n".join(text_parts)
                print(f"   ✅ {len(self.text)} caractères extraits")

                return self.text

        except Exception as e:
            print(f"❌ Erreur lors de la lecture du PDF: {e}")
            raise

class DocumentExtractor:
    """Extracteur de texte avec parsing structuré"""

    def __init__(self, text: str):
        self.raw_text = text
        self.sections: List[DocumentSection] = []
        self.tables: List[Dict] = []
        self.recommendations: List[Dict] = []

    def clean_text(self, text: str) -> str:
        #Nettoie le texte
        # Normalisation Unicode
        text = unicodedata.normalize('NFKC', text)

        # Suppression des sauts de page multiples
        text = re.sub(r'\n{3,}', '\n\n', text)

        # Nettoyage des espaces
        text = re.sub(r' +', ' ', text)
        text = re.sub(r'\t+', ' ', text)

        return text.strip()

    def extract_sections(self) -> List[DocumentSection]:
        """Extrait les sections principales du document"""
        cleaned_text = self.clean_text(self.raw_text)

        # Patterns pour identifier les sections
        patterns = [
            (r'---\s*PAGE\s+(\d+)\s*---', 'page_marker'),
            (r'^\d+\s*(?:ère|e)\s*PARTIE\s*:\s*(.+?)$', 'partie'),
            (r'^([IVX]+)\.\s+(.+?)$', 'section_1'),
            (r'^([A-Z])\.\s+(.+?)$', 'section_2'),
            (r'Recommandation\s*N°?\s*(\d+)\s*:\s*(.+?)$', 'recommandation'),
            (r'Tableau\s+(\d+)\.\s*(.+?)$', 'tableau'),
            (r'Graphique\s+(\d+)\.\s*(.+?)$', 'graphique'),
        ]

        lines = cleaned_text.split('\n')
        current_section = None
        current_content = []
        current_page = 1
        temp_sections = []  # Liste temporaire pour stocker les sections avant numérotation

        for i, line in enumerate(lines):
            line_stripped = line.strip()

            if not line_stripped:
                if current_content:
                    current_content.append('')
                continue

            # Vérifier les marqueurs de page
            page_match = re.match(r'---\s*PAGE\s+(\d+)\s*---', line_stripped)
            if page_match:
                current_page = int(page_match.group(1))
                continue

            # Détection des sections
            section_found = False

            for pattern, section_type in patterns:
                if section_type == 'page_marker':
                    continue

                match = re.match(pattern, line_stripped, re.IGNORECASE)
                if match:
                    # Sauvegarder la section précédente
                    if current_section and current_content:
                        current_section.content = '\n'.join(current_content).strip()
                        if current_section.content:  # Ne garder que si contenu non vide
                            temp_sections.append(current_section)

                    # Créer nouvelle section (ID sera assigné plus tard)
                    current_section = DocumentSection(
                        id="",  # Sera rempli après
                        title=line_stripped,
                        content="",
                        section_type=section_type,
                        page_number=current_page,
                        metadata={'line_number': i}
                    )
                    current_content = []
                    section_found = True
                    break

            if not section_found:
                current_content.append(line_stripped)

        # Ajouter la dernière section
        if current_section and current_content:
            current_section.content = '\n'.join(current_content).strip()
            if current_section.content:
                temp_sections.append(current_section)

        # Renuméroter les sections de manière séquentielle
        for idx, section in enumerate(temp_sections, 1):
            section.id = f"sec_{idx}"
            self.sections.append(section)

        # Si aucune section détectée, créer des sections par blocs de texte
        if not self.sections:
            print("   ⚠️  Aucune section structurée détectée, création de sections par blocs...")
            self._create_fallback_sections(cleaned_text)

        return self.sections

    def _create_fallback_sections(self, text: str):
        """Crée des sections par découpage simple si pas de structure détectée"""
        paragraphs = text.split('\n\n')
        current_page = 1
        section_counter = 0

        for i, para in enumerate(paragraphs):
            if len(para.strip()) > 50:  # Ignorer les paragraphes trop courts
                # Détecter les changements de page
                page_match = re.search(r'PAGE\s+(\d+)', para)
                if page_match:
                    current_page = int(page_match.group(1))

                section_counter += 1
                # Créer une section pour chaque paragraphe significatif
                title = para[:100].strip() + "..."
                self.sections.append(DocumentSection(
                    id=f"sec_{section_counter}",
                    title=title,
                    content=para.strip(),
                    section_type='paragraph',
                    page_number=current_page,
                    metadata={'paragraph_index': i}
                ))

    def extract_tables(self) -> List[Dict]:
        """Extrait les tableaux du document"""
        table_pattern = r'Tableau\s+(\d+)\.\s*(.+?)(?=\n{2,}|\Z)'

        matches = re.finditer(table_pattern, self.raw_text, re.MULTILINE | re.DOTALL)

        for match in matches:
            table_num = match.group(1)
            table_content = match.group(2).strip()

            # Extraire le titre (première ligne)
            lines = table_content.split('\n')
            title = lines[0] if lines else f"Tableau {table_num}"
            content = '\n'.join(lines[1:]) if len(lines) > 1 else table_content

            self.tables.append({
                'id': f'table_{table_num}',
                'number': table_num,
                'title': title,
                'content': content,
                'type': 'table'
            })

        return self.tables

    def extract_recommendations(self) -> List[Dict]:
        """Extrait toutes les recommandations"""
        pattern = r'Recommandation\s*N°?\s*(\d+)\s*:\s*(.+?)(?=\n\n|Recommandation|$)'

        matches = re.finditer(pattern, self.raw_text, re.MULTILINE | re.DOTALL)

        for match in matches:
            rec_num = match.group(1)
            rec_content = match.group(2).strip()

            self.recommendations.append({
                'id': f'rec_{rec_num}',
                'number': int(rec_num),
                'content': rec_content,
                'type': 'recommendation'
            })

        return self.recommendations

    def extract_key_statistics(self) -> Dict:
        """Extrait les statistiques clés du document"""
        stats = {}

        # Extraction des ratios S/P
        sp_pattern = r'S/P.*?(\d+)%'
        sp_matches = re.findall(sp_pattern, self.raw_text)
        if sp_matches:
            stats['sp_ratios'] = [int(m) for m in sp_matches[:10]]

        # Extraction des montants en MDT
        mdt_pattern = r'(\d+(?:[,\.]\d+)?)\s*(?:MDT|millions de dinars)'
        mdt_matches = re.findall(mdt_pattern, self.raw_text, re.IGNORECASE)
        if mdt_matches:
            stats['amounts_mdt'] = mdt_matches[:10]

        # Extraction des années
        year_pattern = r'\b(20\d{2})\b'
        years = set(re.findall(year_pattern, self.raw_text))
        stats['years_covered'] = sorted(list(years))

        return stats

    def get_document_structure(self) -> Dict:
        """Retourne la structure complète du document"""
        return {
            'total_sections': len(self.sections),
            'total_tables': len(self.tables),
            'total_recommendations': len(self.recommendations),
            'sections': [asdict(s) for s in self.sections],
            'tables': self.tables,
            'recommendations': self.recommendations
        }


class ChunkManager:
    """Gestionnaire de découpage du texte pour RAG"""

    def __init__(self, chunk_size: int = 1000, overlap: int = 200):
        self.chunk_size = chunk_size
        self.overlap = overlap

    def create_chunks(self, sections: List[DocumentSection]) -> List[Dict]:
        """Crée des chunks avec contexte"""
        chunks = []
        chunk_id = 0

        for section in sections:
            # Si la section est courte, la garder entière
            if len(section.content) <= self.chunk_size:
                chunks.append({
                    'id': f'chunk_{chunk_id}',
                    'content': section.content,
                    'title': section.title,
                    'section_type': section.section_type,
                    'metadata': {
                        'section_id': section.id,
                        'page': section.page_number,
                        'is_complete': True
                    }
                })
                chunk_id += 1
            else:
                # Découper la section avec overlap
                text = section.content
                start = 0
                part = 1

                while start < len(text):
                    end = start + self.chunk_size
                    chunk_text = text[start:end]

                    chunks.append({
                        'id': f'chunk_{chunk_id}',
                        'content': chunk_text,
                        'title': f"{section.title[:50]}... (partie {part})",
                        'section_type': section.section_type,
                        'metadata': {
                            'section_id': section.id,
                            'page': section.page_number,
                            'is_complete': False,
                            'part': part
                        }
                    })

                    chunk_id += 1
                    part += 1
                    start = end - self.overlap

        return chunks


def main():

    # Chemin vers le PDF
    pdf_path = "rapport_rc_auto_tunisie.pdf"



    # Extraction du PDF
    pdf_extractor = PDFExtractor(pdf_path)
    try:
        pdf_text = pdf_extractor.extract_text()
    except Exception as e:
        print(f"\n❌ Erreur: {e}")
        return None

    # Initialisation de l'extracteur
    print("\n🔍 Analyse du document...")
    extractor = DocumentExtractor(pdf_text)

    # Extraction des sections
    print("📄 Extraction des sections...")
    sections = extractor.extract_sections()
    print(f"   ✓ {len(sections)} sections extraites")

    # Vérification de la numérotation séquentielle
    print("   🔢 Vérification de la numérotation...")
    for i, section in enumerate(sections[:5], 1):
        expected_id = f"sec_{i}"
        if section.id != expected_id:
            print(f"   ⚠️  Erreur: section {section.id} au lieu de {expected_id}")
        else:
            print(f"   ✓ {section.id}: {section.title[:60]}...")

    # Extraction des tableaux
    print("\n📊 Extraction des tableaux...")
    tables = extractor.extract_tables()
    print(f"   ✓ {len(tables)} tableaux extraits")

    # Extraction des recommandations
    print("💡 Extraction des recommandations...")
    recommendations = extractor.extract_recommendations()
    print(f"   ✓ {len(recommendations)} recommandations extraites")

    # Extraction des statistiques
    print("📈 Extraction des statistiques...")
    stats = extractor.extract_key_statistics()
    print(f"   ✓ Statistiques extraites")

    # Création des chunks
    print("✂️  Création des chunks...")
    chunk_manager = ChunkManager(chunk_size=1000, overlap=200)
    chunks = chunk_manager.create_chunks(sections)
    print(f"   ✓ {len(chunks)} chunks créés")

    # Sauvegarde des résultats
    output = {
        'document_info': {
            'title': 'Rapport RC Auto Tunisie',
            'filename': pdf_path,
            'date': 'Août 2015',
            'total_pages': pdf_extractor.page_count,
            'total_characters': len(pdf_text)
        },
        'structure': extractor.get_document_structure(),
        'statistics': stats,
        'chunks': chunks
    }

    # Export JSON
    output_path = Path('extracted_data1.json')
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(output, f, ensure_ascii=False, indent=2)

    # Export du texte brut
    text_output_path = Path('extracted_text1.txt')
    with open(text_output_path, 'w', encoding='utf-8') as f:
        f.write(pdf_text)

    print(f"\n{'='*60}")
    print(f"✅ EXTRACTION TERMINÉE!")
    print(f"{'='*60}")
    print(f"📁 Données JSON: {output_path}")
    print(f"📁 Texte brut: {text_output_path}")
    print(f"\n📊 RÉSUMÉ:")
    print(f"   • Pages: {pdf_extractor.page_count}")
    print(f"   • Caractères: {len(pdf_text):,}")
    print(f"   • Sections: {len(sections)}")
    print(f"   • Tableaux: {len(tables)}")
    print(f"   • Recommandations: {len(recommendations)}")
    print(f"   • Chunks: {len(chunks)}")

    if stats.get('years_covered'):
        print(f"   • Années couvertes: {', '.join(map(str, stats['years_covered']))}")

    # Afficher les 5 premières sections pour vérification
    print(f"\n📋 Aperçu des sections (5 premières):")
    for section in sections[:5]:
        print(f"   {section.id}: {section.title[:70]}...")

    return output


if __name__ == "__main__":
    main()

📖 Lecture du PDF: rapport_rc_auto_tunisie.pdf
   📄 68 pages détectées
   ⏳ Extraction en cours... 10/68 pages
   ⏳ Extraction en cours... 20/68 pages
   ⏳ Extraction en cours... 30/68 pages
   ⏳ Extraction en cours... 40/68 pages
   ⏳ Extraction en cours... 50/68 pages
   ⏳ Extraction en cours... 60/68 pages
   ✅ 163223 caractères extraits

🔍 Analyse du document...
📄 Extraction des sections...
   ✓ 150 sections extraites
   🔢 Vérification de la numérotation...
   ✓ sec_1: 1èrePartie : Diagnostic de l’assurance Responsabilité Civile...
   ✓ sec_2: I. La tarification de la RC Auto est inadaptée et pénalise l...
   ✓ sec_3: B. La sous -tarification de la RC obligatoire est compensée ...
   ✓ sec_4: E. Le déséquilibre de la RC Auto est moins prononcé dans le ...
   ✓ sec_5: D. Pourtant la mise en œuvre rigoureuse d’un tel système pré...

📊 Extraction des tableaux...
   ✓ 13 tableaux extraits
💡 Extraction des recommandations...
   ✓ 33 recommandations extraites
📈 Extraction des statistiques

In [10]:
"""
Système RAG Hybride pour l'Analyse de Documents d'Assurance
Phase 2: Création des Embeddings et Indexation Hybride (BM25 + Vector)
"""

import json
import numpy as np
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import pickle
from collections import defaultdict
import math

try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    print("❌ sentence-transformers non installé")
    print("   Installez avec: pip install sentence-transformers")
    exit(1)

try:
    from rank_bm25 import BM25Okapi
except ImportError:
    print("❌ rank-bm25 non installé")
    print("   Installez avec: pip install rank-bm25")
    exit(1)

try:
    import faiss
except ImportError:
    print("⚠️  FAISS non installé (optionnel pour grandes bases)")
    print("   Installez avec: pip install faiss-cpu")
    faiss = None


@dataclass
class SearchResult:
    """Résultat de recherche enrichi"""
    chunk_id: str
    content: str
    score: float
    metadata: Dict
    rank: int
    search_type: str  # 'bm25', 'semantic', 'hybrid'


class EmbeddingManager:
    """Gestionnaire des embeddings vectoriels"""

    def __init__(self, model_name: str = "paraphrase-multilingual-mpnet-base-v2"):
        """
        Initialise le modèle d'embeddings

        Modèles recommandés:
        - paraphrase-multilingual-mpnet-base-v2 (bon rapport qualité/vitesse)
        - intfloat/multilingual-e5-large (meilleur mais plus lourd)
        - dangvantuan/sentence-camembert-large (français optimisé)
        """
        print(f"🤖 Chargement du modèle: {model_name}")
        self.model = SentenceTransformer(model_name)
        self.model_name = model_name
        self.dimension = self.model.get_sentence_embedding_dimension()
        print(f"   ✓ Dimension des embeddings: {self.dimension}")

    def create_embeddings(self, texts: List[str], batch_size: int = 32) -> np.ndarray:
        """Crée les embeddings pour une liste de textes"""
        print(f"🔄 Création des embeddings pour {len(texts)} textes...")

        embeddings = self.model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_numpy=True,
            normalize_embeddings=True  # Normalisation pour cosine similarity
        )

        print(f"   ✓ Shape: {embeddings.shape}")
        return embeddings

    def embed_query(self, query: str) -> np.ndarray:
        """Crée l'embedding pour une requête"""
        embedding = self.model.encode(
            query,
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        return embedding


class BM25Index:
    """Index BM25 pour recherche par mots-clés"""

    def __init__(self):
        self.bm25 = None
        self.tokenized_corpus = []
        self.documents = []

    def build_index(self, documents: List[Dict]):
        """Construit l'index BM25"""
        print("📚 Construction de l'index BM25...")

        self.documents = documents

        # Tokenisation simple (peut être améliorée avec spaCy/NLTK)
        self.tokenized_corpus = [
            self._tokenize(doc['content'])
            for doc in documents
        ]

        self.bm25 = BM25Okapi(self.tokenized_corpus)
        print(f"   ✓ Index BM25 créé avec {len(documents)} documents")

    def _tokenize(self, text: str) -> List[str]:
        """Tokenisation basique (à améliorer selon besoin)"""
        # Conversion en minuscules et split
        tokens = text.lower().split()

        # Nettoyage basique
        tokens = [t.strip('.,;:!?()[]{}\"\'') for t in tokens]
        tokens = [t for t in tokens if len(t) > 2]  # Ignorer mots trop courts

        return tokens

    def search(self, query: str, top_k: int = 10) -> List[Tuple[int, float]]:
        """Recherche BM25"""
        tokenized_query = self._tokenize(query)
        scores = self.bm25.get_scores(tokenized_query)

        # Récupérer les top-k indices
        top_indices = np.argsort(scores)[::-1][:top_k]
        results = [(idx, scores[idx]) for idx in top_indices]

        return results

    def save(self, path: str):
        """Sauvegarde l'index"""
        with open(path, 'wb') as f:
            pickle.dump({
                'bm25': self.bm25,
                'tokenized_corpus': self.tokenized_corpus,
                'documents': self.documents
            }, f)

    def load(self, path: str):
        """Charge l'index"""
        with open(path, 'rb') as f:
            data = pickle.load(f)
            self.bm25 = data['bm25']
            self.tokenized_corpus = data['tokenized_corpus']
            self.documents = data['documents']


class VectorIndex:
    """Index vectoriel pour recherche sémantique"""

    def __init__(self, dimension: int, use_faiss: bool = True):
        self.dimension = dimension
        self.use_faiss = use_faiss and faiss is not None
        self.embeddings = None
        self.documents = []
        self.faiss_index = None

    def build_index(self, embeddings: np.ndarray, documents: List[Dict]):
        """Construit l'index vectoriel"""
        print("🔍 Construction de l'index vectoriel...")

        self.embeddings = embeddings.astype('float32')
        self.documents = documents

        if self.use_faiss:
            # Index FAISS pour grandes bases de données
            self.faiss_index = faiss.IndexFlatIP(self.dimension)  # Inner Product (cosine avec vecteurs normalisés)
            self.faiss_index.add(self.embeddings)
            print(f"   ✓ Index FAISS créé avec {self.faiss_index.ntotal} vecteurs")
        else:
            print(f"   ✓ Index NumPy créé avec {len(self.embeddings)} vecteurs")

    def search(self, query_embedding: np.ndarray, top_k: int = 10) -> List[Tuple[int, float]]:
        """Recherche par similarité vectorielle"""
        query_embedding = query_embedding.astype('float32').reshape(1, -1)

        if self.use_faiss:
            # Recherche FAISS
            scores, indices = self.faiss_index.search(query_embedding, top_k)
            results = [(int(idx), float(score)) for idx, score in zip(indices[0], scores[0])]
        else:
            # Recherche NumPy (cosine similarity)
            scores = np.dot(self.embeddings, query_embedding.T).flatten()
            top_indices = np.argsort(scores)[::-1][:top_k]
            results = [(int(idx), float(scores[idx])) for idx in top_indices]

        return results

    def save(self, path: str):
        """Sauvegarde l'index"""
        data = {
            'embeddings': self.embeddings,
            'documents': self.documents,
            'dimension': self.dimension
        }

        if self.use_faiss:
            faiss.write_index(self.faiss_index, f"{path}_faiss.index")

        with open(f"{path}_data.pkl", 'wb') as f:
            pickle.dump(data, f)

    def load(self, path: str):
        """Charge l'index"""
        with open(f"{path}_data.pkl", 'rb') as f:
            data = pickle.load(f)
            self.embeddings = data['embeddings']
            self.documents = data['documents']
            self.dimension = data['dimension']

        if self.use_faiss and Path(f"{path}_faiss.index").exists():
            self.faiss_index = faiss.read_index(f"{path}_faiss.index")


class HybridSearchEngine:
    """Moteur de recherche hybride BM25 + Vectoriel"""

    def __init__(self, bm25_index: BM25Index, vector_index: VectorIndex,
                 embedding_manager: EmbeddingManager):
        self.bm25_index = bm25_index
        self.vector_index = vector_index
        self.embedding_manager = embedding_manager

    def search(self, query: str, top_k: int = 5,
               bm25_weight: float = 0.4, semantic_weight: float = 0.6,
               search_mode: str = 'hybrid') -> List[SearchResult]:
        """
        Recherche hybride avec pondération

        Args:
            query: Requête utilisateur
            top_k: Nombre de résultats
            bm25_weight: Poids de la recherche par mots-clés
            semantic_weight: Poids de la recherche sémantique
            search_mode: 'hybrid', 'bm25', 'semantic'
        """
        results = []

        if search_mode in ['hybrid', 'bm25']:
            # Recherche BM25
            bm25_results = self.bm25_index.search(query, top_k=top_k*2)
            bm25_scores = {idx: score for idx, score in bm25_results}
        else:
            bm25_scores = {}

        if search_mode in ['hybrid', 'semantic']:
            # Recherche sémantique
            query_embedding = self.embedding_manager.embed_query(query)
            semantic_results = self.vector_index.search(query_embedding, top_k=top_k*2)
            semantic_scores = {idx: score for idx, score in semantic_results}
        else:
            semantic_scores = {}

        # Fusion des scores
        all_indices = set(bm25_scores.keys()) | set(semantic_scores.keys())

        combined_scores = {}
        for idx in all_indices:
            # Normalisation des scores
            bm25_score = self._normalize_score(bm25_scores.get(idx, 0), list(bm25_scores.values()))
            semantic_score = self._normalize_score(semantic_scores.get(idx, 0), list(semantic_scores.values()))

            # Score hybride pondéré
            if search_mode == 'hybrid':
                combined_scores[idx] = (bm25_weight * bm25_score +
                                       semantic_weight * semantic_score)
            elif search_mode == 'bm25':
                combined_scores[idx] = bm25_score
            else:  # semantic
                combined_scores[idx] = semantic_score

        # Trier et créer les résultats
        sorted_indices = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

        for rank, (idx, score) in enumerate(sorted_indices, 1):
            doc = self.bm25_index.documents[idx]

            results.append(SearchResult(
                chunk_id=doc['id'],
                content=doc['content'],
                score=score,
                metadata=doc.get('metadata', {}),
                rank=rank,
                search_type=search_mode
            ))

        return results

    def _normalize_score(self, score: float, all_scores: List[float]) -> float:
        """Normalise un score entre 0 et 1"""
        if not all_scores or max(all_scores) == 0:
            return 0.0
        return score / max(all_scores)

    def rerank_with_context(self, results: List[SearchResult],
                           query: str) -> List[SearchResult]:
        """Re-ranking avec prise en compte du contexte"""
        # Bonus pour les résultats avec métadonnées pertinentes
        for result in results:
            bonus = 0.0

            # Bonus si contient des recommandations
            if result.metadata.get('has_recommendations'):
                bonus += 0.1

            # Bonus si contient des chiffres (pour questions statistiques)
            if result.metadata.get('has_numbers') and any(c.isdigit() for c in query):
                bonus += 0.1

            # Bonus si section principale (niveau hiérarchique élevé)
            if result.metadata.get('section_type') in ['partie', 'section_1']:
                bonus += 0.05

            result.score += bonus

        # Re-trier
        results.sort(key=lambda x: x.score, reverse=True)

        # Mettre à jour les rangs
        for rank, result in enumerate(results, 1):
            result.rank = rank

        return results


class RAGPipeline:
    """Pipeline RAG complet"""

    def __init__(self, extracted_data_path: str = "extracted_data1.json",
                 model_name: str = "paraphrase-multilingual-mpnet-base-v2"):

        self.extracted_data_path = extracted_data_path
        self.model_name = model_name

        self.embedding_manager = None
        self.bm25_index = None
        self.vector_index = None
        self.search_engine = None
        self.chunks = []

    def build_indexes(self):



        # Charger les données extraites
        print("\n📂 Chargement des données extraites...")
        with open(self.extracted_data_path, 'r', encoding='utf-8') as f:
            data = json.load(f)


        if 'chunks' in data:
        # Ancienne structure
          self.chunks = data['chunks']

        elif 'structure' in data and 'sections' in data['structure']:
            # Transformer les sections en chunks
            self.chunks = [
            {
                'id': sec['id'],
                'content': sec.get('content', ''),
                'metadata': {
                    'page': sec.get('page_number'),
                    **sec.get('metadata', {}),
                    'section_type': sec.get('section_type', '')
                }
            }
            for sec in data['structure']['sections']
        ]
        else:
          raise ValueError("Le JSON fourni n'a pas de format reconnu (ni 'chunks' ni 'structure/sections').")

        print(f"   ✓ {len(self.chunks)} chunks chargés")

    # Initialiser le gestionnaire d'embeddings
        self.embedding_manager = EmbeddingManager(self.model_name)

    # Créer les embeddings
        print("\n🎯 Création des embeddings...")
        texts = [chunk['content'] for chunk in self.chunks]
        embeddings = self.embedding_manager.create_embeddings(texts)

    # Construire l'index BM25
        self.bm25_index = BM25Index()
        self.bm25_index.build_index(self.chunks)

    # Construire l'index vectoriel
        self.vector_index = VectorIndex(
            dimension=self.embedding_manager.dimension,
            use_faiss=(faiss is not None and len(self.chunks) > 1000)
        )
        self.vector_index.build_index(embeddings, self.chunks)

    # Créer le moteur de recherche
        self.search_engine = HybridSearchEngine(
            self.bm25_index,
            self.vector_index,
            self.embedding_manager
        )

        print("\n✅ Index créés avec succès!")

    def save_indexes(self, output_dir: str = "rag_indexes"):
        """Sauvegarde tous les index"""
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)

        print(f"\n💾 Sauvegarde des index dans {output_dir}/...")

        # Sauvegarder BM25
        self.bm25_index.save(output_path / "bm25_index.pkl")
        print("   ✓ Index BM25 sauvegardé")

        # Sauvegarder Vector Index
        self.vector_index.save(str(output_path / "vector_index"))
        print("   ✓ Index vectoriel sauvegardé")

        # Sauvegarder métadonnées
        metadata = {
            'model_name': self.model_name,
            'num_chunks': len(self.chunks),
            'embedding_dimension': self.embedding_manager.dimension
        }

        with open(output_path / "metadata.json", 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2)

        print("   ✓ Métadonnées sauvegardées")
        print(f"\n✅ Tous les index sauvegardés dans {output_dir}/")

    def load_indexes(self, input_dir: str = "rag_indexes"):
        """Charge les index depuis le disque"""
        input_path = Path(input_dir)

        print(f"📥 Chargement des index depuis {input_dir}...")

        # Charger métadonnées
        with open(input_path / "metadata.json", 'r') as f:
            metadata = json.load(f)

        self.model_name = metadata['model_name']

        # Initialiser le gestionnaire d'embeddings
        self.embedding_manager = EmbeddingManager(self.model_name)

        # Charger BM25
        self.bm25_index = BM25Index()
        self.bm25_index.load(input_path / "bm25_index.pkl")

        # Charger Vector Index
        self.vector_index = VectorIndex(
            dimension=metadata['embedding_dimension'],
            use_faiss=True
        )
        self.vector_index.load(str(input_path / "vector_index"))

        # Créer le moteur de recherche
        self.search_engine = HybridSearchEngine(
            self.bm25_index,
            self.vector_index,
            self.embedding_manager
        )

        self.chunks = self.bm25_index.documents

        print("✅ Index chargés avec succès!")

    def search(self, query: str, top_k: int = 5,
               search_mode: str = 'hybrid', use_reranking: bool = True) -> List[SearchResult]:
        """Interface de recherche"""

        if not self.search_engine:
            raise ValueError("Index non initialisés. Appelez build_indexes() ou load_indexes() d'abord.")

        results = self.search_engine.search(query, top_k=top_k, search_mode=search_mode)

        if use_reranking:
            results = self.search_engine.rerank_with_context(results, query)

        return results

    def print_results(self, results: List[SearchResult], query: str):
        """Affiche les résultats de recherche"""
        print("\n" + "="*60)
        print(f"🔍 RÉSULTATS POUR: {query}")
        print("="*60)

        for result in results:
            print(f"\n📄 Rang {result.rank} | Score: {result.score:.4f} | Type: {result.search_type}")
            print(f"   Chunk ID: {result.chunk_id}")
            print(f"   Page: {result.metadata.get('page', 'N/A')}")
            print(f"   Section: {result.metadata.get('section_type', 'N/A')}")
            print(f"\n   Contenu:")
            print(f"   {result.content[:300]}...")
            print("-"*60)


def main():
    """Fonction principale - Construction des index"""

    # Créer le pipeline
    pipeline = RAGPipeline(
        extracted_data_path="extracted_data1.json",
        model_name="paraphrase-multilingual-mpnet-base-v2"
    )

    # Construire les index
    pipeline.build_indexes()

    # Sauvegarder
    pipeline.save_indexes("rag_indexes")

    # Tests de recherche
    print("\n" + "="*60)
    print("🧪 TESTS DE RECHERCHE")
    print("="*60)

    test_queries = [
        "Quel est le ratio S/P moyen en Tunisie?",
        "Quelles sont les recommandations principales?",
        "Statistiques sur les accidents de la route",
        "Évolution des primes d'assurance"
    ]

    for query in test_queries:
        results = pipeline.search(query, top_k=3, search_mode='hybrid')
        pipeline.print_results(results, query)
        print("\n")




if __name__ == "__main__":
      main()


📂 Chargement des données extraites...
   ✓ 256 chunks chargés
🤖 Chargement du modèle: paraphrase-multilingual-mpnet-base-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   ✓ Dimension des embeddings: 768

🎯 Création des embeddings...
🔄 Création des embeddings pour 256 textes...


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

   ✓ Shape: (256, 768)
📚 Construction de l'index BM25...
   ✓ Index BM25 créé avec 256 documents
🔍 Construction de l'index vectoriel...
   ✓ Index NumPy créé avec 256 vecteurs

✅ Index créés avec succès!

💾 Sauvegarde des index dans rag_indexes/...
   ✓ Index BM25 sauvegardé
   ✓ Index vectoriel sauvegardé
   ✓ Métadonnées sauvegardées

✅ Tous les index sauvegardés dans rag_indexes/

🧪 TESTS DE RECHERCHE

🔍 RÉSULTATS POUR: Quel est le ratio S/P moyen en Tunisie?

📄 Rang 1 | Score: 0.8249 | Type: hybrid
   Chunk ID: chunk_86
   Page: 18
   Section: N/A

   Contenu:
   Pays Ecart de
prime en % Ratio
max / min Prime la plus
favorable
après...années Nombre de
décès/100 000
véhicules
Allemagne 245-30 8,2 21 8,9
Suisse 240-35 7,5 13 6,9
France 350-50 7,0 13 9,0
Tunisie
87,0
Usage personnel 350-70 5,0 10
Usage professionnel 200-80 2,5
Source : réglementations nationale...
------------------------------------------------------------

📄 Rang 2 | Score: 0.8216 | Type: hybrid
   Chunk ID: chunk_1

In [ ]:
import openai

client = openai.OpenAI(
    api_key="" ,
    base_url="https://api.groq.com/openai/v1"
)

# Lister les modèles
models = client.models.list()
for model in models.data:
    print(model.id)

moonshotai/kimi-k2-instruct-0905
meta-llama/llama-prompt-guard-2-86m
llama-3.3-70b-versatile
openai/gpt-oss-120b
qwen/qwen3-32b
whisper-large-v3-turbo
playai-tts
meta-llama/llama-guard-4-12b
groq/compound
meta-llama/llama-4-scout-17b-16e-instruct
allam-2-7b
moonshotai/kimi-k2-instruct
meta-llama/llama-prompt-guard-2-22m
whisper-large-v3
playai-tts-arabic
meta-llama/llama-4-maverick-17b-128e-instruct
openai/gpt-oss-20b
llama-3.1-8b-instant
groq/compound-mini


In [ ]:
import json
import numpy as np
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import pickle
from datetime import datetime
import hashlib

# Installation automatique des dépendances
def install_dependencies():
    """Installe les dépendances nécessaires sur Colab"""
    import subprocess
    import sys

    packages = [
        'sentence-transformers',
        'rank-bm25',
        'gradio',
        'openai',
        'nltk',
        'rouge-score',
    ]

    print("📦 Installation des dépendances...")
    for package in packages:
        try:
            __import__(package.replace('-', '_'))
        except ImportError:
            print(f"   Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print("✅ Dépendances installées\n")

# Décommenter pour installation automatique sur Colab
install_dependencies()

try:
    from sentence_transformers import SentenceTransformer
    from rank_bm25 import BM25Okapi
    import gradio as gr
    import openai
    import nltk
    from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
    from rouge_score import rouge_scorer
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
    exit(1)

# Télécharger les données NLTK nécessaires
try:
    nltk.download('punkt', quiet=True)
except:
    pass

GROQ_API_KEY = ""
GROQ_MODEL = "llama-3.1-8b-instant"


# ============================================================
# NOUVELLES CLASSES POUR L'ÉVALUATION
# ============================================================

@dataclass
class EvaluationMetrics:
    """Métriques d'évaluation du système RAG"""
    cosine_similarity: float
    bleu_score: float
    rouge_1: float
    rouge_2: float
    rouge_l: float
    query: str
    timestamp: str


class RAGEvaluator:
    """Évaluateur pour le système RAG"""

    def __init__(self, embedding_manager):
        self.embedding_manager = embedding_manager
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1

    def evaluate_retrieval(self, query: str, retrieved_docs: List[SearchResult]) -> float:
        """
        Évalue la qualité de la récupération avec Cosine Similarity
        Calcule la similarité moyenne entre la requête et les documents récupérés
        """
        if not retrieved_docs:
            return 0.0

        # Embedding de la requête
        query_embedding = self.embedding_manager.embed_query(query)

        # Embeddings des documents récupérés
        doc_texts = [doc.content for doc in retrieved_docs]
        doc_embeddings = self.embedding_manager.create_embeddings(doc_texts)

        # Calcul des cosine similarities
        similarities = []
        for doc_emb in doc_embeddings:
            similarity = np.dot(query_embedding, doc_emb) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb)
            )
            similarities.append(similarity)

        # Moyenne des similarités
        avg_cosine_similarity = np.mean(similarities)
        return float(avg_cosine_similarity)

    def evaluate_generation_bleu(self, reference: str, candidate: str) -> float:
        """
        Évalue la génération avec BLEU score
        """
        reference_tokens = reference.lower().split()
        candidate_tokens = candidate.lower().split()

        if not candidate_tokens:
            return 0.0

        # BLEU score avec smoothing
        bleu = sentence_bleu(
            [reference_tokens],
            candidate_tokens,
            smoothing_function=self.smoothing
        )
        return float(bleu)

    def evaluate_generation_rouge(self, reference: str, candidate: str) -> Dict[str, float]:
        """
        Évalue la génération avec ROUGE scores (ROUGE-1, ROUGE-2, ROUGE-L)
        """
        scores = self.rouge_scorer.score(reference, candidate)

        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }

    def evaluate_full(self, query: str, retrieved_docs: List[SearchResult],
                      generated_answer: str, reference_answer: str) -> EvaluationMetrics:
        """
        Évaluation complète: récupération + génération
        """
        # Évaluation de la récupération
        cosine_sim = self.evaluate_retrieval(query, retrieved_docs)

        # Évaluation de la génération
        bleu = self.evaluate_generation_bleu(reference_answer, generated_answer)
        rouge_scores = self.evaluate_generation_rouge(reference_answer, generated_answer)

        return EvaluationMetrics(
            cosine_similarity=cosine_sim,
            bleu_score=bleu,
            rouge_1=rouge_scores['rouge1'],
            rouge_2=rouge_scores['rouge2'],
            rouge_l=rouge_scores['rougeL'],
            query=query,
            timestamp=datetime.now().isoformat()
        )


# ============================================================
# CLASSES ORIGINALES (inchangées)
# ============================================================

@dataclass
class SearchResult:
    """Résultat de recherche enrichi"""
    chunk_id: str
    content: str
    score: float
    metadata: Dict
    rank: int
    search_type: str


@dataclass
class RAGResponse:
    """Réponse générée par le système RAG"""
    answer: str
    sources: List[SearchResult]
    query: str
    timestamp: str
    confidence: float


class QueryCache:
    """Cache des requêtes pour performances"""

    def __init__(self, max_size: int = 100):
        self.cache = {}
        self.max_size = max_size
        self.hits = 0
        self.misses = 0

    def _hash_query(self, query: str) -> str:
        """Crée un hash de la requête"""
        return hashlib.md5(query.lower().strip().encode()).hexdigest()

    def get(self, query: str) -> Optional[RAGResponse]:
        """Récupère une réponse en cache"""
        key = self._hash_query(query)
        if key in self.cache:
            self.hits += 1
            return self.cache[key]
        self.misses += 1
        return None

    def set(self, query: str, response: RAGResponse):
        """Stocke une réponse en cache"""
        if len(self.cache) >= self.max_size:
            oldest = next(iter(self.cache))
            del self.cache[oldest]

        key = self._hash_query(query)
        self.cache[key] = response

    def get_stats(self) -> Dict:
        """Statistiques du cache"""
        total = self.hits + self.misses
        hit_rate = self.hits / total if total > 0 else 0
        return {
            'hits': self.hits,
            'misses': self.misses,
            'hit_rate': f"{hit_rate:.2%}",
            'cache_size': len(self.cache)
        }


class EmbeddingManager:
    """Gestionnaire des embeddings vectoriels"""

    def __init__(self, model_name: str = "paraphrase-multilingual-mpnet-base-v2"):
        print(f"🤖 Chargement du modèle: {model_name}")
        self.model = SentenceTransformer(model_name)
        self.model_name = model_name
        self.dimension = self.model.get_sentence_embedding_dimension()
        print(f"   ✓ Dimension: {self.dimension}")

    def create_embeddings(self, texts: List[str], batch_size: int = 32) -> np.ndarray:
        """Crée les embeddings pour une liste de textes"""
        embeddings = self.model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        return embeddings

    def embed_query(self, query: str) -> np.ndarray:
        """Crée l'embedding pour une requête"""
        embedding = self.model.encode(
            query,
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        return embedding


class BM25Index:
    """Index BM25 pour recherche par mots-clés"""

    def __init__(self):
        self.bm25 = None
        self.tokenized_corpus = []
        self.documents = []

    def build_index(self, documents: List[Dict]):
        """Construit l'index BM25"""
        self.documents = documents
        self.tokenized_corpus = [
            self._tokenize(doc['content'])
            for doc in documents
        ]
        self.bm25 = BM25Okapi(self.tokenized_corpus)

    def _tokenize(self, text: str) -> List[str]:
        """Tokenisation simple"""
        tokens = text.lower().split()
        tokens = [t.strip('.,;:!?()[]{}\"\'') for t in tokens]
        tokens = [t for t in tokens if len(t) > 2]
        return tokens

    def search(self, query: str, top_k: int = 10) -> List[Tuple[int, float]]:
        """Recherche BM25"""
        tokenized_query = self._tokenize(query)
        scores = self.bm25.get_scores(tokenized_query)
        top_indices = np.argsort(scores)[::-1][:top_k]
        results = [(idx, scores[idx]) for idx in top_indices]
        return results

    def save(self, path: str):
        with open(path, 'wb') as f:
            pickle.dump({
                'bm25': self.bm25,
                'tokenized_corpus': self.tokenized_corpus,
                'documents': self.documents
            }, f)

    def load(self, path: str):
        with open(path, 'rb') as f:
            data = pickle.load(f)
            self.bm25 = data['bm25']
            self.tokenized_corpus = data['tokenized_corpus']
            self.documents = data['documents']


class VectorIndex:
    """Index vectoriel pour recherche sémantique"""

    def __init__(self, dimension: int):
        self.dimension = dimension
        self.embeddings = None
        self.documents = []

    def build_index(self, embeddings: np.ndarray, documents: List[Dict]):
        """Construit l'index vectoriel"""
        self.embeddings = embeddings.astype('float32')
        self.documents = documents

    def search(self, query_embedding: np.ndarray, top_k: int = 10) -> List[Tuple[int, float]]:
        """Recherche par similarité vectorielle"""
        query_embedding = query_embedding.astype('float32').reshape(1, -1)
        scores = np.dot(self.embeddings, query_embedding.T).flatten()
        top_indices = np.argsort(scores)[::-1][:top_k]
        results = [(int(idx), float(scores[idx])) for idx in top_indices]
        return results

    def save(self, path: str):
        data = {
            'embeddings': self.embeddings,
            'documents': self.documents,
            'dimension': self.dimension
        }
        with open(f"{path}_data.pkl", 'wb') as f:
            pickle.dump(data, f)

    def load(self, path: str):
        with open(f"{path}_data.pkl", 'rb') as f:
            data = pickle.load(f)
            self.embeddings = data['embeddings']
            self.documents = data['documents']
            self.dimension = data['dimension']


class HybridSearchEngine:
    """Moteur de recherche hybride BM25 + Vectoriel"""

    def __init__(self, bm25_index: BM25Index, vector_index: VectorIndex,
                 embedding_manager: EmbeddingManager):
        self.bm25_index = bm25_index
        self.vector_index = vector_index
        self.embedding_manager = embedding_manager

    def search(self, query: str, top_k: int = 5,
               bm25_weight: float = 0.4, semantic_weight: float = 0.6) -> List[SearchResult]:
        """Recherche hybride avec pondération"""

        # Recherche BM25
        bm25_results = self.bm25_index.search(query, top_k=top_k*2)
        bm25_scores = {idx: score for idx, score in bm25_results}

        # Recherche sémantique
        query_embedding = self.embedding_manager.embed_query(query)
        semantic_results = self.vector_index.search(query_embedding, top_k=top_k*2)
        semantic_scores = {idx: score for idx, score in semantic_results}

        # Fusion des scores
        all_indices = set(bm25_scores.keys()) | set(semantic_scores.keys())

        combined_scores = {}
        for idx in all_indices:
            bm25_score = self._normalize_score(bm25_scores.get(idx, 0), list(bm25_scores.values()))
            semantic_score = self._normalize_score(semantic_scores.get(idx, 0), list(semantic_scores.values()))

            combined_scores[idx] = (bm25_weight * bm25_score + semantic_weight * semantic_score)

        sorted_indices = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

        results = []
        for rank, (idx, score) in enumerate(sorted_indices, 1):
            doc = self.bm25_index.documents[idx]

            results.append(SearchResult(
                chunk_id=doc['id'],
                content=doc['content'],
                score=score,
                metadata=doc.get('metadata', {}),
                rank=rank,
                search_type='hybrid'
            ))

        return results

    def _normalize_score(self, score: float, all_scores: List[float]) -> float:
        """Normalise un score entre 0 et 1"""
        if not all_scores or max(all_scores) == 0:
            return 0.0
        return score / max(all_scores)


class GroqLLMGenerator:
    """Générateur de réponses avec Groq API"""

    def __init__(self, api_key: str, model: str = "llama-3.1-8b-instant"):
        """Initialise le générateur avec Groq API"""

        if not api_key or api_key == "votre_cle_api_groq_ici":
            raise ValueError(
                "❌ Clé API Groq manquante!\n"
                "Obtenez une clé gratuite sur: https://console.groq.com\n"
                "Puis modifiez la variable GROQ_API_KEY dans le code."
            )

        self.model = model
        self.client = openai.OpenAI(
            api_key=api_key,
            base_url="https://api.groq.com/openai/v1"
        )

        print(f"✅ Groq API connectée avec modèle: {model}")

    def generate(self, query: str, context: List[SearchResult],
                 max_tokens: int = 800) -> str:
        """Génère une réponse basée sur le contexte"""

        # Construire le contexte
        context_text = self._build_context(context)

        # Construire le prompt
        prompt = self._build_prompt(query, context_text)

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {
                        "role": "system",
                        "content": "Tu es un assistant expert en analyse de documents d'assurance automobile en Tunisie. Tu réponds de manière précise, professionnelle et concise."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                max_tokens=max_tokens,
                temperature=0.3
            )

            return response.choices[0].message.content

        except Exception as e:
            return f"❌ Erreur lors de la génération: {str(e)}\n\nVérifiez votre clé API Groq."

    def _build_context(self, results: List[SearchResult], max_length: int = 3500) -> str:
        """Construit le contexte depuis les résultats de recherche"""
        context_parts = []
        current_length = 0

        for result in results:
            content = result.content[:600]
            part = f"[Source {result.rank} - Page {result.metadata.get('page', 'N/A')}]\n{content}\n"

            if current_length + len(part) > max_length:
                break

            context_parts.append(part)
            current_length += len(part)

        return "\n".join(context_parts)

    def _build_prompt(self, query: str, context: str) -> str:
        """Construit le prompt pour le LLM"""
        prompt = f"""Tu es un assistant expert en analyse de documents d'assurance automobile en Tunisie.

CONTEXTE DOCUMENTAIRE:
{context}

QUESTION DE L'UTILISATEUR:
{query}

INSTRUCTIONS:
- Réponds de manière précise et concise en te basant UNIQUEMENT sur le contexte fourni
- Cite les pages sources entre parenthèses (ex: "selon la page 18")
- Si l'information n'est pas dans le contexte, dis-le clairement
- Utilise un langage professionnel mais accessible
- Structure ta réponse avec des paragraphes clairs
- Inclus les chiffres et statistiques pertinents disponibles dans le contexte

RÉPONSE:"""

        return prompt


class RAGSystem:
    """Système RAG complet"""

    def __init__(self, extracted_data_path: str = "extracted_data.json",
                 model_name: str = "paraphrase-multilingual-mpnet-base-v2",
                 groq_api_key: str = None,
                 groq_model: str = "llama-3.1-8b-instant"):

        self.extracted_data_path = extracted_data_path
        self.model_name = model_name

        self.embedding_manager = None
        self.bm25_index = None
        self.vector_index = None
        self.search_engine = None
        self.llm_generator = None
        self.query_cache = QueryCache(max_size=100)
        self.evaluator = None  # NOUVEAU
        self.chunks = []

        # Statistiques
        self.stats = {
            'total_queries': 0,
            'cache_hits': 0,
            'avg_response_time': 0
        }

        # Initialiser le LLM avec Groq
        self.llm_generator = GroqLLMGenerator(api_key=groq_api_key, model=groq_model)

    def build_indexes(self):
        """Construit tous les index"""
        print("="*60)
        print("🚀 CONSTRUCTION DU SYSTÈME RAG COMPLET")
        print("="*60)

        # Charger les données
        print("\n📂 Chargement des données...")
        with open(self.extracted_data_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Adaptation pour différents formats JSON
        if 'chunks' in data:
            self.chunks = data['chunks']
        elif 'structure' in data and 'sections' in data['structure']:
            self.chunks = [
                {
                    'id': sec['id'],
                    'content': sec.get('content', ''),
                    'metadata': {
                        'page': sec.get('page_number'),
                        **sec.get('metadata', {}),
                        'section_type': sec.get('section_type', '')
                    }
                }
                for sec in data['structure']['sections']
            ]
        else:
            raise ValueError("Format JSON non reconnu. Le fichier doit contenir 'chunks' ou 'structure/sections'.")

        print(f"   ✓ {len(self.chunks)} chunks chargés")

        # Embeddings
        self.embedding_manager = EmbeddingManager(self.model_name)
        print("\n🎯 Création des embeddings...")
        texts = [chunk['content'] for chunk in self.chunks]
        embeddings = self.embedding_manager.create_embeddings(texts)

        # Index BM25
        print("\n📚 Construction index BM25...")
        self.bm25_index = BM25Index()
        self.bm25_index.build_index(self.chunks)
        print("   ✓ Index BM25 prêt")

        # Index vectoriel
        print("\n🔍 Construction index vectoriel...")
        self.vector_index = VectorIndex(dimension=self.embedding_manager.dimension)
        self.vector_index.build_index(embeddings, self.chunks)
        print("   ✓ Index vectoriel prêt")

        # Moteur de recherche
        self.search_engine = HybridSearchEngine(
            self.bm25_index,
            self.vector_index,
            self.embedding_manager
        )

        # Initialiser l'évaluateur - NOUVEAU
        self.evaluator = RAGEvaluator(self.embedding_manager)
        print("   ✓ Évaluateur initialisé")

        print("\n✅ Système RAG prêt!")

    def query(self, question: str, top_k: int = 5, use_cache: bool = True) -> RAGResponse:
        """Interface principale de requête"""

        start_time = datetime.now()
        self.stats['total_queries'] += 1

        # Vérifier le cache
        if use_cache:
            cached = self.query_cache.get(question)
            if cached:
                self.stats['cache_hits'] += 1
                print("⚡ Réponse depuis le cache")
                return cached

        # Recherche hybride
        print(f"🔍 Recherche hybride...")
        results = self.search_engine.search(question, top_k=top_k)

        # Génération avec Groq
        print("🤖 Génération de la réponse avec Groq...")
        answer = self.llm_generator.generate(question, results)

        # Calculer la confiance
        confidence = np.mean([r.score for r in results]) if results else 0.0

        # Créer la réponse
        response = RAGResponse(
            answer=answer,
            sources=results,
            query=question,
            timestamp=datetime.now().isoformat(),
            confidence=confidence
        )

        # Mettre en cache
        if use_cache:
            self.query_cache.set(question, response)

        # Stats
        elapsed = (datetime.now() - start_time).total_seconds()
        self.stats['avg_response_time'] = (
            (self.stats['avg_response_time'] * (self.stats['total_queries'] - 1) + elapsed)
            / self.stats['total_queries']
        )

        print(f"✅ Réponse générée en {elapsed:.2f}s")

        return response

    # NOUVELLE MÉTHODE D'ÉVALUATION
    def evaluate(self, query: str, reference_answer: str, top_k: int = 5) -> EvaluationMetrics:
        """
        Évalue le système RAG sur une requête

        Args:
            query: La question posée
            reference_answer: La réponse de référence pour comparaison
            top_k: Nombre de documents à récupérer

        Returns:
            EvaluationMetrics contenant toutes les métriques d'évaluation
        """
        # Récupération
        retrieved_docs = self.search_engine.search(query, top_k=top_k)

        # Génération
        generated_answer = self.llm_generator.generate(query, retrieved_docs)

        # Évaluation complète
        metrics = self.evaluator.evaluate_full(
            query=query,
            retrieved_docs=retrieved_docs,
            generated_answer=generated_answer,
            reference_answer=reference_answer
        )

        return metrics

    def get_stats(self) -> Dict:
        """Retourne les statistiques du système"""
        cache_stats = self.query_cache.get_stats()
        return {
            **self.stats,
            **cache_stats,
            'num_chunks': len(self.chunks)
        }

    def save_indexes(self, output_dir: str = "rag_indexes"):
        """Sauvegarde les index"""
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)

        print(f"\n💾 Sauvegarde dans {output_dir}/...")
        self.bm25_index.save(output_path / "bm25_index.pkl")
        self.vector_index.save(str(output_path / "vector_index"))

        metadata = {
            'model_name': self.model_name,
            'num_chunks': len(self.chunks),
            'embedding_dimension': self.embedding_manager.dimension
        }

        with open(output_path / "metadata.json", 'w') as f:
            json.dump(metadata, f, indent=2)

        print("✅ Index sauvegardés")

    def load_indexes(self, input_dir: str = "rag_indexes"):
        """Charge les index depuis le disque"""
        input_path = Path(input_dir)

        print(f"📥 Chargement depuis {input_dir}...")

        with open(input_path / "metadata.json", 'r') as f:
            metadata = json.load(f)

        self.model_name = metadata['model_name']
        self.embedding_manager = EmbeddingManager(self.model_name)

        self.bm25_index = BM25Index()
        self.bm25_index.load(input_path / "bm25_index.pkl")

        self.vector_index = VectorIndex(dimension=metadata['embedding_dimension'])
        self.vector_index.load(str(input_path / "vector_index"))

        self.search_engine = HybridSearchEngine(
            self.bm25_index,
            self.vector_index,
            self.embedding_manager
        )

        # Initialiser l'évaluateur - NOUVEAU
        self.evaluator = RAGEvaluator(self.embedding_manager)

        self.chunks = self.bm25_index.documents

        print("✅ Index chargés!")


def create_gradio_interface(rag_system: RAGSystem):
    """Crée l'interface Gradio simplifiée"""

    def query_interface(question):
        """Fonction appelée par Gradio"""
        if not question.strip():
            return "⚠️ Veuillez entrer une question", ""

        try:
            response = rag_system.query(question, top_k=5)

            # Formater la réponse
            answer = response.answer

            # Formater les sources
            sources_text = "### 📚 Sources documentaires:\n\n"
            for r in response.sources:
                sources_text += f"**Source {r.rank}** (Pertinence: {r.score:.2%}, Page: {r.metadata.get('page', 'N/A')})\n"
                sources_text += f"```\n{r.content[:400]}...\n```\n\n"

            return answer, sources_text

        except Exception as e:
            error_msg = f"❌ Erreur: {str(e)}"
            return error_msg, ""

    # NOUVELLE FONCTION POUR L'ÉVALUATION
    def evaluate_interface(query, reference_answer):
        """Évalue le système RAG"""
        if not query.strip() or not reference_answer.strip():
            return "⚠️ Veuillez fournir une question et une réponse de référence"

        try:
            metrics = rag_system.evaluate(query, reference_answer, top_k=5)

            results = f"""### 📊 Résultats d'Évaluation

**Question:** {query}

#### 🔍 Évaluation de la Récupération
- **Cosine Similarity:** {metrics.cosine_similarity:.4f}

#### 📝 Évaluation de la Génération
- **BLEU Score:** {metrics.bleu_score:.4f}
- **ROUGE-1:** {metrics.rouge_1:.4f}
- **ROUGE-2:** {metrics.rouge_2:.4f}
- **ROUGE-L:** {metrics.rouge_l:.4f}

**Timestamp:** {metrics.timestamp}
"""
            return results

        except Exception as e:
            return f"❌ Erreur: {str(e)}"

    # Interface Gradio avec onglet d'évaluation
    with gr.Blocks(title="RAG Assurance Tunisie", theme=gr.themes.Soft()) as interface:

        gr.Markdown("""
        # 🚗 Système RAG - Assurance Automobile Tunisie

        **Analyse intelligente de documents d'assurance avec recherche hybride et IA générative**
        """)

        with gr.Tabs():
            # ONGLET 1: Requêtes normales
            with gr.Tab("💬 Requêtes"):
                gr.Markdown("""
                Posez vos questions sur le rapport d'assurance RC Auto en Tunisie. Le système combine:
                - 🔍 Recherche par mots-clés (BM25)
                - 🧠 Recherche sémantique (embeddings)
                - 🤖 Génération de réponses (Groq AI)
                """)

                with gr.Row():
                    with gr.Column(scale=3):
                        question_input = gr.Textbox(
                            label="❓ Posez votre question",
                            placeholder="Ex: Quel est le ratio S/P moyen en Tunisie?",
                            lines=3
                        )

                        submit_btn = gr.Button("🔍 Rechercher et Générer", variant="primary", size="lg")

                    with gr.Column(scale=1):
                        gr.Markdown("""
                        ### 💡 Exemples:

                        - Ratio S/P moyen
                        - Recommandations principales
                        - Statistiques accidents
                        - Évolution des primes
                        - Système bonus-malus
                        - Situation des deux-roues
                        """)

                gr.Markdown("---")

                answer_output = gr.Markdown(
                    label="📝 Réponse générée",
                    value="*La réponse apparaîtra ici...*"
                )

                gr.Markdown("---")

                sources_output = gr.Markdown(
                    label="📚 Sources",
                    value="*Les sources documentaires apparaîtront ici...*"
                )

                # Actions
                submit_btn.click(
                    fn=query_interface,
                    inputs=[question_input],
                    outputs=[answer_output, sources_output]
                )

                # Exemples cliquables
                gr.Examples(
                    examples=[
                        ["Quel est le ratio S/P moyen en Tunisie?"],
                        ["Quelles sont les recommandations principales du rapport?"],
                        ["Statistiques sur les accidents de la route en Tunisie"],
                        ["Problèmes du système bonus-malus actuel"],
                        ["Situation des véhicules deux-roues dans l'assurance"],
                    ],
                    inputs=[question_input]
                )

            # ONGLET 2: Évaluation
            with gr.Tab("📊 Évaluation"):
                gr.Markdown("""
                ### Évaluez la performance du système RAG

                Fournissez une question et une réponse de référence pour calculer:
                - **Cosine Similarity**: Qualité de la récupération des documents
                - **BLEU**: Qualité de la génération (n-grams)
                - **ROUGE-1, ROUGE-2, ROUGE-L**: Qualité de la génération (recall)
                """)

                with gr.Row():
                    with gr.Column():
                        eval_query = gr.Textbox(
                            label="❓ Question",
                            placeholder="Ex: Quel est le ratio S/P moyen?",
                            lines=2
                        )

                        eval_reference = gr.Textbox(
                            label="✅ Réponse de Référence",
                            placeholder="Ex: Le ratio S/P moyen en Tunisie est de 75%...",
                            lines=5
                        )

                        eval_btn = gr.Button("📊 Évaluer", variant="primary", size="lg")

                gr.Markdown("---")

                eval_output = gr.Markdown(
                    label="📊 Résultats d'Évaluation",
                    value="*Les métriques d'évaluation apparaîtront ici...*"
                )

                eval_btn.click(
                    fn=evaluate_interface,
                    inputs=[eval_query, eval_reference],
                    outputs=[eval_output]
                )

                # Exemples d'évaluation
                gr.Examples(
                    examples=[
                        [
                            "Quel est le ratio S/P moyen en Tunisie?",
                            "Le ratio S/P (sinistres sur primes) moyen en Tunisie est de 75%, ce qui indique que pour chaque dinar de prime collectée, 0.75 dinars sont utilisés pour payer les sinistres."
                        ],
                        [
                            "Quels sont les principaux problèmes du système bonus-malus?",
                            "Les principaux problèmes du système bonus-malus actuel sont le manque de transparence, l'absence d'actualisation régulière des tarifs, et la difficulté de suivi pour les assurés qui changent de compagnie."
                        ]
                    ],
                    inputs=[eval_query, eval_reference]
                )

    return interface


def main():
    """Fonction principale"""

    print("="*60)
    print("🚀 SYSTÈME RAG COMPLET - DÉMARRAGE")
    print("="*60)

    # Initialiser le système avec Groq
    rag_system = RAGSystem(
        extracted_data_path="extracted_data.json",
        model_name="paraphrase-multilingual-mpnet-base-v2",
        groq_api_key=GROQ_API_KEY,
        groq_model=GROQ_MODEL
    )

    # Construire ou charger les index
    if Path("rag_indexes/metadata.json").exists():
        rag_system.load_indexes()
    else:
        rag_system.build_indexes()
        rag_system.save_indexes()

    # Test CLI
    print("\n" + "="*60)
    print("🧪 TEST EN LIGNE DE COMMANDE")
    print("="*60)

    test_question = "Quel est le ratio S/P moyen en Tunisie?"
    print(f"\n❓ Question: {test_question}")

    response = rag_system.query(test_question, top_k=3)

    print("\n📝 Réponse:")
    print(response.answer)

    print("\n📚 Sources:")
    for src in response.sources:
        print(f"- Source {src.rank} (Score: {src.score:.3f}, Page: {src.metadata.get('page', 'N/A')})")
        print(f"  {src.content[:150]}...")

    # Test d'évaluation
    print("\n" + "="*60)
    print("📊 TEST D'ÉVALUATION")
    print("="*60)

    test_reference = "Le ratio S/P moyen en Tunisie est de 75%, ce qui indique une bonne gestion des sinistres."

    print(f"\n❓ Question: {test_question}")
    print(f"\n✅ Référence: {test_reference}")

    metrics = rag_system.evaluate(test_question, test_reference, top_k=3)

    print("\n📊 Métriques d'Évaluation:")
    print(f"  • Cosine Similarity: {metrics.cosine_similarity:.4f}")
    print(f"  • BLEU Score: {metrics.bleu_score:.4f}")
    print(f"  • ROUGE-1: {metrics.rouge_1:.4f}")
    print(f"  • ROUGE-2: {metrics.rouge_2:.4f}")
    print(f"  • ROUGE-L: {metrics.rouge_l:.4f}")

    # Lancer l'interface Gradio
    print("\n" + "="*60)
    print("🌐 LANCEMENT DE L'INTERFACE GRADIO")
    print("="*60)

    interface = create_gradio_interface(rag_system)
    interface.launch(share=True)

    print("\n✅ Interface Gradio lancée!")


if __name__ == "__main__":
    main()

📦 Installation des dépendances...
   Installing rouge-score...
✅ Dépendances installées

🚀 SYSTÈME RAG COMPLET - DÉMARRAGE
✅ Groq API connectée avec modèle: llama-3.1-8b-instant
📥 Chargement depuis rag_indexes...
🤖 Chargement du modèle: paraphrase-multilingual-mpnet-base-v2
   ✓ Dimension: 768
✅ Index chargés!

🧪 TEST EN LIGNE DE COMMANDE

❓ Question: Quel est le ratio S/P moyen en Tunisie?
🔍 Recherche hybride...
🤖 Génération de la réponse avec Groq...
✅ Réponse générée en 0.84s

📝 Réponse:
La question du ratio S/P moyen en Tunisie est abordée dans le contexte fourni, mais elle n'est pas explicitement mentionnée. Cependant, nous pouvons déduire cette information en analysant les données disponibles.

Selon la page 18 du contexte (Source 1), nous trouvons les informations suivantes :

- La prime la plus favorable après 10 ans sans accident est de 5,0 % pour l'usage personnel.
- La prime la plus favorable après 10 ans sans accident est de 2,5 % pour l'usage professionnel.

Puisque le rat

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📊 Métriques d'Évaluation:
  • Cosine Similarity: 0.6181
  • BLEU Score: 0.0208
  • ROUGE-1: 0.0659
  • ROUGE-2: 0.0442
  • ROUGE-L: 0.0659

🌐 LANCEMENT DE L'INTERFACE GRADIO
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5fadc23da2980f20d6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



✅ Interface Gradio lancée!
